管线命令不经常用，但是很有用。它们可以把多个步骤组合成一个对象执行。这样
可以更方便灵活地调节和控制整个模型的配置，而不只是一个一个步骤调节。  
这里我们首先通过计算处理缺失值；然后将数据集调整为均值为
0，标准差为1的标准形。

In [37]:
from sklearn import datasets as d
import numpy as np 

In [38]:
mat = d.make_spd_matrix(10)
mat[:4,:4]

array([[ 0.95115386,  1.2985632 ,  0.75860609,  0.05532325],
       [ 1.2985632 ,  3.85065463,  2.04004338,  0.5969275 ],
       [ 0.75860609,  2.04004338,  1.46376199,  0.40184326],
       [ 0.05532325,  0.5969275 ,  0.40184326,  0.66975123]])

> 不使用管线命令时的数据预处理

In [47]:
# 处理缺失值
masking_array = np.random.binomial(1,.1,mat.shape).astype(bool)
mat[masking_array] = 1
mat[:4,:4]

array([[ nan,  nan,  nan,  nan],
       [ nan,  10.,  nan,  nan],
       [ nan,  nan,  nan,   1.],
       [  1.,   1.,  nan,  nan]])

In [48]:
from sklearn import preprocessing
impute = preprocessing.Imputer()
scaler = preprocessing.StandardScaler()
# 处理缺失值
mat_imputed = impute.fit_transform(mat)
mat_imputed[:4,:4]

array([[  4.6,   5.5,   1. ,   1. ],
       [  4.6,  10. ,   1. ,   1. ],
       [  4.6,   5.5,   1. ,   1. ],
       [  1. ,   1. ,   1. ,   1. ]])

In [50]:
# 标准化
mat_imp_and_scaled=scaler.fit_transform(mat_imputed)
mat_imp_and_scaled[:4,:4]

array([[ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  1.58113883,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [-1.15470054, -1.58113883,  0.        ,  0.        ]])

> 使用pipeline

In [51]:
from sklearn import pipeline
pipe = pipeline.Pipeline([('impute',impute),('scaler',scaler)])

In [54]:
pipe
# 管线命令定义了处理步骤

Pipeline(steps=[('impute', Imputer(axis=0, copy=True, missing_values='NaN', strategy='mean', verbose=0)), ('scaler', StandardScaler(copy=True, with_mean=True, with_std=True))])

In [55]:
new_mat = pipe.fit_transform(mat)
new_mat[:4,:4]

array([[ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  1.58113883,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [-1.15470054, -1.58113883,  0.        ,  0.        ]])

In [56]:
np.array_equal(new_mat,mat_imp_and_scaled)

True

管线命令不仅可以用于预处理步骤中，在降维、算法拟合中也可以很方便的使用。和其他算法接口类似，pipeline中最重要的三个函数：  
* fit  
* transform  
* fit_transform  
它就是一组 	for	循环，对每个步骤执
行 	fit	和 	transform	，然后把结果传递到下一个变换操作中。
使用管线命令的理由主要有两点：
首先是方便。代码会简洁一些，不需要重复调用 	fit	和 	transform	。  
其次，也是更重要的作用，就是使用交叉验证。模型可以变得很复杂。如果管
线命令中的一个步骤调整了参数，那么它们必然需要重新测试；测试一个步骤
参数的代码管理成本是很低的。但是，如果测试5个步骤的全部参数会变都很
复杂。管线命令可以缓解这些负担。